In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://0x0.st/H5DC.csv")
df

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."
...,...,...,...,...
70891,9,14,Stan,I think you're pushing it.\n
70892,9,14,Randy,How about twenty?\n
70893,9,14,Stan,That's not disciprine.\n
70894,9,14,Randy,Right right. Does vodka count?\n


In [3]:
import re

REGEX_WHITESPACE = re.compile(r"\s+")


def clean_text(text):
    text = re.sub(REGEX_WHITESPACE, " ", text)
    text = text.strip()
    return text

In [4]:
df["cleaned_text"] = df["Line"].apply(clean_text)

In [9]:
df.sort_values(by=['Season', 'Episode'], inplace=True)

df['ep_id'] = df['Season'].astype(str) + "-" + df['Episode'].astype(str)

df['prev_line'] = df.groupby('ep_id')['cleaned_text'].shift(1)
df['prev_char'] = df.groupby('ep_id')['Character'].shift(1)

stan_df = df[(df['Character'] == 'Stan') & df['prev_line'].notnull()]

stan_df['context'] = stan_df['prev_char'] + ": " + stan_df['prev_line']

qa_df = stan_df[['context', 'cleaned_text']].rename(columns={'context': 'question', 'cleaned_text': 'answer'}).reset_index(drop=True)

qa_df

/tmp/ipykernel_58001/1242685049.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stan_df['context'] = stan_df['prev_char'] + ": " + stan_df['prev_line']


,question,answer
0,"Kyle: Dude, don't call my brother a dildo!",What's a dildo?
1,Cartman: I'm not telling you.,"What's a dildo, Kenny?"
2,"Cartman: He-yeah, that's what Kyle's little br...","Dude, that kicks ass!"
3,Kyle: Kick the baby.,"Whoa, Cartman! Looks like you didn't get much ..."
4,"Cartman: Well, I dreamt that I was lying in my...",Dude! Visitors!
...,...,...
7646,Kyle: I don't wanna know when I'm going to die...,"Guys, guys, guys, stop! Look at what this thin..."
7647,Jason: How could be nothing but trouble?,We risked everything to get it from the girls....
7648,Craig: Maybe we should take it so someplace sa...,And then what? Hide it forever from the govern...
7649,Cartman: Are you guys nuts?! After everything ...,We got it away from the girls. That's what mat...


In [11]:
from datasets import Dataset

qa_dataset = Dataset.from_pandas(qa_df)
qa_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 7651
})

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def prompt(character, text):
    return f"{character}: {text}"

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], return_tensors="pt", padding=True, truncation=True)